This notebook is used to do a main model run

In [1]:
from epimodel import EpidemiologicalParameters, preprocess_data
from epimodel.pymc3_models.models import ComplexDifferentEffectsModel

import numpy as np
import pymc3 as pm

KeyboardInterrupt: 

In [ ]:
data = preprocess_data('../merged_data/data_final_nov.csv', last_day='2020-05-30', smoothing=1)
data.mask_reopenings(print_out = False)

In [ ]:
ep = EpidemiologicalParameters()

In [ ]:
bd = ep.get_model_build_dict()

In [ ]:
bd

In [ ]:
with ComplexDifferentEffectsModel(data) as model:
    model.build_model(**bd)

In [ ]:
pm.model_to_graphviz(model)

In [ ]:
with model:
    model.trace = pm.sample(2000, tune=500, cores=4, chains=4, max_treedepth=12, target_accept=0.925)

In [ ]:
# save results in a pickle file
import pickle
pickle.dump(model.trace, open('traces/final_nov.pkl', 'wb'))

In [ ]:
# csv file for mitigation calculator
cm_red = np.zeros((8000, 8))
cm_red[:, :] = model.trace.CMReduction[:, [0, 1, 2, 3, 4, 5, 6, 8]]

# combine schools and unis into one NPI
cm_red[:, 6] = cm_red[:, 6] * model.trace.CMReduction[:, 7]

np.savetxt('traces.txt', cm_red, delimiter=',', header='Mask Wearing, Gatherings<1000, Gatherings<100, Gatherings<10, Some Business, Most Business, School+Uni, StayHome' )